In [1]:
import sys
sys.path.append("..")  # Add the parent directory of 'examples' to the Python path


## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [2]:
from multiversx_sdk.core import Address

address = Address.new_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

print("Address (bech32-encoded)", address.to_bech32())
print("Public key (hex-encoded):", address.to_hex())
print("Public key (hex-encoded):", address.pubkey.hex())

Address (bech32-encoded) erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [3]:
address = Address.new_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1", "erd")

... or from a raw public key:

In [4]:
pubkey = bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = Address(pubkey, "erd")

Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [5]:
from multiversx_sdk.core import AddressFactory

factory = AddressFactory("erd")

address = factory.create_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
address = factory.create_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = factory.create_from_public_key(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))

Addresses can be converted from one representation to another as follows:

In [6]:
print(address.to_bech32())
print(address.to_hex())

erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


Getting the shard of an address:

In [7]:
from multiversx_sdk.core import AddressComputer

address_computer = AddressComputer(number_of_shards=3)
print("Shard:", address_computer.get_shard_of_address(address))

Shard: 1


Checking whether an address is a smart contract:

In [8]:
address = Address.new_from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

print("Is contract:", address.is_smart_contract())

Is contract: True


## EGLD / ESDT transfers

Create an EGLD transfer:

In [9]:
from multiversx_sdk.core import Transaction

transaction = Transaction(
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000
)

print(transaction.__dict__)

{'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 50000, 'nonce': 77, 'value': 1000000000000000000, 'data': b'', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}


In case you are using a **guarded** account you should also populate the `guardian` and `guardian_signature` fields after creating the transaction.

We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an EGLD transfer, but this time with a payload (data):

In [10]:
transaction = Transaction(
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    gas_limit=50000,
    chain_id="D",
    nonce=77,
    value=1000000000000000000,
    data=b"for the book"
)

print(transaction.__dict__)

{'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 50000, 'nonce': 77, 'value': 1000000000000000000, 'data': b'for the book', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}


Alternatively, we can create an EGLD transfer using a **transaction factory** (as we will see below, transaction factories are more commonly used). But before that, we have to create a configuration object (for any factory that we might use):

In [11]:
from multiversx_sdk.core.transaction_factories import TransactionsFactoryConfig

config = TransactionsFactoryConfig(chain_id="D")

The **transaction factory** is parametrized at instantiation, and the transaction is obtained by invoking the `create_transaction...` method:

In [12]:
from multiversx_sdk.core import TokenComputer
from multiversx_sdk.core.transaction_factories import \
    TransferTransactionsFactory

transfer_factory = TransferTransactionsFactory(config, TokenComputer())
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")

# With "data" field
transaction = transfer_factory.create_transaction_for_native_token_transfer(
    sender=alice,
    receiver=bob,
    native_amount=1000000000000000000,
    data="for the book"
)

print("Transaction:", transaction.__dict__)
print("Transaction data:", transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 68000, 'nonce': 0, 'value': 1000000000000000000, 'data': b'for the book', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'for the book'


Create a single ESDT transfer:

In [13]:
from multiversx_sdk.core import Token, TokenTransfer

token = Token("TEST-8b028f")
transfer = TokenTransfer(token, 10000)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction.__dict__)
print("Transaction data:", transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 410000, 'nonce': 0, 'value': 0, 'data': b'ESDTTransfer@544553542d386230323866@2710', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'ESDTTransfer@544553542d386230323866@2710'


Create a single NFT transfer:

Keep in mind, since we are sending a NFT, we `should` set the amount to `1`.

In [14]:
token = Token(identifier="TEST-38f249", nonce=1)
transfer = TokenTransfer(token=token, amount=1)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction.__dict__)
print("Transaction data:", transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'gas_limit': 1213500, 'nonce': 0, 'value': 0, 'data': b'ESDTNFTTransfer@544553542d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'ESDTNFTTransfer@544553542d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8'


Create a single SFT transfer (almost the same as above, the only difference being that for the transfer we set the desired amount, as an integer):

In [15]:
token = Token(identifier="SEMI-9efd0f", nonce=1)
transfer = TokenTransfer(token=token, amount=5)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[transfer]
)

print("Transaction:", transaction.__dict__)
print("Transaction data:", transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'gas_limit': 1213500, 'nonce': 0, 'value': 0, 'data': b'ESDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'ESDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8'


Create a multiple ESDT / NFT transfer:

In [16]:
first_token = Token(identifier="TEST-38f249", nonce=1)
first_transfer = TokenTransfer(token=first_token, amount=1)

second_token = Token(identifier="BAR-c80d29")
second_transfer = TokenTransfer(token=second_token, amount=10000000000000000000)

transaction = transfer_factory.create_transaction_for_esdt_token_transfer(
    sender=alice,
    receiver=bob,
    token_transfers=[first_transfer, second_transfer]
)

print("Transaction:", transaction.__dict__)
print("Transaction data:", transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'gas_limit': 1484000, 'nonce': 0, 'value': 0, 'data': b'MultiESDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'MultiESDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000'


## Relayed Transactions

First, we get the newtwork configuration using the network providers.

In [17]:
from multiversx_sdk.network_providers import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")
network_config = provider.get_network_config()

### Relayed V1

In [18]:
from pathlib import Path

from multiversx_sdk.core import Address, Transaction, TransactionComputer
from multiversx_sdk.core.transaction_factories.relayed_transactions_factory import \
    RelayedTransactionsFactory
from multiversx_sdk.core.transaction_factories.transactions_factory_config import \
    TransactionsFactoryConfig
from multiversx_sdk.wallet.user_signer import UserSigner

signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

inner_tx = Transaction(
    chain_id=network_config.chain_id,
    sender="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    receiver="erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z",
    gas_limit=60000000,
    nonce=198,
    data=b"add@05"
)
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

relayed_tx = factory.create_relayed_v1_transaction(
    inner_transaction=inner_tx,
    relayer_address=relayer
)
relayed_tx.nonce = 2627

print(relayed_tx.__dict__)

{'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 61052000, 'nonce': 2627, 'value': 0, 'data': b'relayedTx@7b226e6f6e6365223a3139382c2273656e646572223a2267456e574f65576d6d413063306a6b71764d354241707a61644b46574e534f69417643575163776d4750673d222c227265636569766572223a22414141414141414141414146414159464e2b4d4e5569353867624233352b635667734c59397267656165453d222c2276616c7565223a302c226761735072696365223a313030303030303030302c226761734c696d6974223a36303030303030302c2264617461223a225957526b51444131222c227369676e6174757265223a22474c43344a59354c30756f4a4d7048753448505767474148476f476a7a476136352b754830302f774c515945515230645a4c635134546d7155394b6d515a3554716d4c6e5351345657466f354366793963642b6c43413d3d222c22636861696e4944223a2252413d3d222c2276657273696f6e223a327d', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'versi

### Relayed V2

In [19]:
from pathlib import Path

from multiversx_sdk.core import Address, Transaction, TransactionComputer
from multiversx_sdk.core.transaction_factories.relayed_transactions_factory import \
    RelayedTransactionsFactory
from multiversx_sdk.core.transaction_factories.transactions_factory_config import \
    TransactionsFactoryConfig
from multiversx_sdk.wallet.user_signer import UserSigner

signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/bob.pem"))
transaction_computer = TransactionComputer()

# for the relayedV2 transactions, the gasLimit for the inner transaction should be 0
inner_tx = Transaction(
            chain_id=network_config.chain_id,
            sender="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
            receiver="erd1qqqqqqqqqqqqqpgqqczn0ccd2gh8eqdswln7w9vzctv0dwq7d8ssm4y34z",
            gas_limit=0,
            nonce=15,
            data=b"add@05"
        )
inner_tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(inner_tx))

config = TransactionsFactoryConfig(chain_id="D")
factory = RelayedTransactionsFactory(config=config)
relayer = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

relayed_tx = factory.create_relayed_v2_transaction(
    inner_transaction=inner_tx,
    inner_transaction_gas_limit=60_000_000,
    relayer_address=relayer
)
relayed_tx.nonce = 37

print(relayed_tx.__dict__)

The `serialize()` method is deprecated and will soon be removed


{'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx', 'gas_limit': 60381500, 'nonce': 37, 'value': 0, 'data': b'relayedTxV2@00000000000000000500060537e30d522e7c81b077e7e71582c2d8f6b81e69e1@0f@616464403035@65072a9118db14747a4f1cf857a6e4373e3e9ead6ea6036f5111bf68803c5cadfdc44d37a21b920449b5e83c783975425c3f476020173ac0fc71481646d4a900', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}


## Contract deployments and interactions

Create a transaction to deploy a smart contract:

In [20]:
from pathlib import Path

from multiversx_sdk.core.transaction_factories import \
    SmartContractTransactionsFactory

sc_factory = SmartContractTransactionsFactory(config, TokenComputer())
bytecode = Path("./data/counter.wasm").read_bytes()

deploy_transaction = sc_factory.create_transaction_for_deploy(
    sender=alice,
    bytecode=bytecode,
    arguments=[42, "test"],
    gas_limit=10000000,
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", deploy_transaction.__dict__)
print("Transaction data:", deploy_transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu', 'gas_limit': 10000000, 'nonce': 0, 'value': 0, 'data': b'0061736d0100000001150460037f7f7e017f60027f7f017e60017e0060000002420303656e7611696e74363473746f7261676553746f7265000003656e7610696e74363473746f726167654c6f6164000103656e760b696e74363466696e6973680002030504030303030405017001010105030100020608017f01419088040b072f05066d656d6f7279020004696e6974000309696e6372656d656e7400040964656372656d656e7400050367657400060a8a01041300418088808000410742011080808080001a0b2e01017e4180888080004107418088808000410710818080800042017c22001080808080001a20001082808080000b2e01017e41808880800041074180888080004107108180808000427f7c22001080808080001a20001082808080000b160041808880800041071081808080001082808080000b0b0f01004180080b08434f554e54455200@0500@0506@2a@74657374', 'signature': b'', 'sender_username': '', 'receiver_username': '

Create a transaction to upgrade an existing smart contract:

In [21]:
contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")
bytecode = Path("./data/counter.wasm").read_bytes()

upgrade_transaction = sc_factory.create_transaction_for_upgrade(
    sender=alice,
    contract=contract_address,
    bytecode=bytecode,
    gas_limit=10000000,
    arguments=[42, "test"],
    is_upgradeable=True,
    is_readable=True,
    is_payable=True,
    is_payable_by_sc=True
)

print("Transaction:", upgrade_transaction.__dict__)
print("Transaction data:", upgrade_transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm', 'gas_limit': 10000000, 'nonce': 0, 'value': 0, 'data': b'upgradeContract@0061736d0100000001150460037f7f7e017f60027f7f017e60017e0060000002420303656e7611696e74363473746f7261676553746f7265000003656e7610696e74363473746f726167654c6f6164000103656e760b696e74363466696e6973680002030504030303030405017001010105030100020608017f01419088040b072f05066d656d6f7279020004696e6974000309696e6372656d656e7400040964656372656d656e7400050367657400060a8a01041300418088808000410742011080808080001a0b2e01017e4180888080004107418088808000410710818080800042017c22001080808080001a20001082808080000b2e01017e41808880800041074180888080004107108180808000427f7c22001080808080001a20001082808080000b160041808880800041071081808080001082808080000b0b0f01004180080b08434f554e54455200@0506@2a@74657374', 'signature': b'', 'sender_username': '', 'receiver_u

Create a transaction that invokes a smart contract function:

In [22]:
contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

call_transaction = sc_factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="foo",
    gas_limit=10000000,
    arguments=[42, "test"]
)

print("Transaction:", call_transaction.__dict__)
print("Transaction data:", call_transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm', 'gas_limit': 10000000, 'nonce': 0, 'value': 0, 'data': b'foo@2a@74657374', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'foo@2a@74657374'


Now, let's create a call that also transfers one or more tokens (**transfer & execute**):

In [23]:
first_token = Token("TEST-38f249", 1)
first_transfer = TokenTransfer(first_token, 1)

second_token = Token("BAR-c80d29")
second_transfer = TokenTransfer(second_token, 10000000000000000000)

transfers = [first_transfer, second_transfer]

call_transaction = sc_factory.create_transaction_for_execute(
    sender=alice,
    contract=contract_address,
    function="hello",
    gas_limit=10000000,
    arguments=[42, "test"],
    token_transfers=transfers
)

print("Transaction:", call_transaction.__dict__)
print("Transaction data:", call_transaction.data)

Transaction: {'chain_id': 'D', 'sender': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'receiver': 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th', 'gas_limit': 10000000, 'nonce': 0, 'value': 0, 'data': b'MultiESDTNFTTransfer@00000000000000000500e0b77f1edfb01cb786a39120f02c31ff5fe40aad8974@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@68656c6c6f@2a@74657374', 'signature': b'', 'sender_username': '', 'receiver_username': '', 'gas_price': 1000000000, 'version': 2, 'options': 0, 'guardian': '', 'guardian_signature': b''}
Transaction data: b'MultiESDTNFTTransfer@00000000000000000500e0b77f1edfb01cb786a39120f02c31ff5fe40aad8974@02@544553542d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@68656c6c6f@2a@74657374'


## Contract queries

In order to create a contract query and run it against a network provider (more details about **network providers** can be found below), do as follows:

In [24]:
from multiversx_sdk.core import ContractQueryBuilder
from multiversx_sdk.core.interfaces import IAddress
from multiversx_sdk.network_providers import ApiNetworkProvider

contract: IAddress = Address.from_bech32("erd1qqqqqqqqqqqqqpgqqy34h7he2ya6qcagqre7ur7cc65vt0mxrc8qnudkr4")

builder = ContractQueryBuilder(
    contract=contract,
    function="getSum",
    call_arguments=[],
    caller=alice
)

query = builder.build()

network_provider = ApiNetworkProvider("https://devnet-api.multiversx.com")
response = network_provider.query_contract(query)

print("Return code:", response.return_code)
print("Return data:", response.return_data)

Return code: ok
Return data: ['BQ==']


## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [25]:
from multiversx_sdk.wallet import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

['abstract', 'peanut', 'satoshi', 'math', 'useful', 'sing', 'slogan', 'fat', 'sorry', 'detect', 'glue', 'lava', 'require', 'post', 'hawk', 'since', 'ancient', 'hen', 'suit', 'slide', 'valley', 'expose', 'sibling', 'lend']


The mnemonic can be saved to a keystore file:

In [26]:
from multiversx_sdk.wallet import UserWallet

wallet = UserWallet.from_mnemonic(mnemonic.get_text(), "password")
wallet.save(Path("./output/walletWithMnemonic.json"))

Given a mnemonic, one can derive keypairs:

In [27]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key:", secret_key.hex())
print("Public key:", public_key.hex())

Secret key: ffd9e3bc5ba50a6f93f2bf83fc319def01affee10a4d2aa29c7904f3c95938c7
Public key: ef5a2b0e341b28c410853aae74423d20fcd6ba7bbad0d62af556a9aa2adf04a9


A keypair can be saved as a JSON wallet:

In [28]:
wallet = UserWallet.from_secret_key(secret_key, "password")
wallet.save(Path("./output/wallet.json"), address_hrp="erd")

... or as a PEM wallet (usually not recommended):

In [29]:
from multiversx_sdk.wallet import UserPEM

label = Address(public_key.buffer, "erd").to_bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(Path("./output/wallet.pem"))

## Loading wallets

This is not a very common use-case - you might refer to [signing objects](#signing-objects) instead.

Load a keystore that holds an **encrypted mnemonic** (and perform wallet derivation at the same time):

In [30]:
from multiversx_sdk.wallet import UserWallet

secret_key = UserWallet.load_secret_key(Path("../multiversx_sdk/testutils/testwallets/withDummyMnemonic.json"), "password", address_index=0)
address = secret_key.generate_public_key().to_address("erd")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load a keystore that holds an **encrypted secret** key:

In [31]:
secret_key = UserWallet.load_secret_key(Path("../multiversx_sdk/testutils/testwallets/alice.json"), "password")
address = secret_key.generate_public_key().to_address("erd")

print("Secret key:", secret_key.hex())
print("Address:", address.to_bech32())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Address: erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th


Load the secret key from a PEM file:

In [32]:
from multiversx_sdk.wallet import UserPEM

pem = UserPEM.from_file(Path("../multiversx_sdk/testutils/testwallets/alice.pem"))

print("Secret key:", pem.secret_key.hex())
print("Public key:", pem.public_key.hex())

Secret key: 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key: 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [33]:
from multiversx_sdk.wallet import UserSigner

signer = UserSigner.from_wallet(Path("../multiversx_sdk/testutils/testwallets/alice.json"), "password")

Creating a `UserSigner` from a PEM file:

In [34]:
signer = UserSigner.from_pem_file(Path("../multiversx_sdk/testutils/testwallets/alice.pem"))

Signing a transaction:

In [35]:
from multiversx_sdk.core import Transaction, TransactionComputer

tx = Transaction(
    nonce=90,
    sender="erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th",
    receiver="erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx",
    value=1000000000000000000,
    gas_limit=50000,
    chain_id="D"
)

transaction_computer = TransactionComputer()

tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))
print("Signature:", tx.signature.hex())

Signature: ee6d665981c00258858923bbbe51e1cfe29911716190a3eedc24e2205100bb05faf3d8133fc973b00409aa18708b41488702d635efefe6a24b4a2b7f66267a00


Signing an arbitrary message:

In [36]:
from multiversx_sdk.core import Message, MessageComputer

signer_address = signer.get_pubkey().to_address(hrp="erd")
message = Message(b"hello", address=signer_address)
message_computer = MessageComputer()

message.signature = signer.sign(message_computer.compute_bytes_for_signing(message))

print("Signature:", message.signature.hex())

Signature: 561bc58f1dc6b10de208b2d2c22c9a474ea5e8cabb59c3d3ce06bbda21cc46454aa71a85d5a60442bd7784effa2e062fcb8fb421c521f898abf7f5ec165e5d0f


## Verifying signatures

Creating a `UserVerifier`:

In [38]:
from multiversx_sdk.core import Address
from multiversx_sdk.wallet import UserVerifier

alice = Address.new_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.new_from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")

alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)

Verifying a signature:

In [39]:
from multiversx_sdk.core import MessageComputer, TransactionComputer

transaction_computer = TransactionComputer()
message_computer = MessageComputer()

print(f"Is signature of Alice?", alice_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Alice?", alice_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))
print(f"Is signature of Bob?", bob_verifier.verify(transaction_computer.compute_bytes_for_signing(tx), tx.signature))
print(f"Is signature of Bob?", bob_verifier.verify(message_computer.compute_bytes_for_verifying(message), message.signature))

Is signature of Alice? True
Is signature of Alice? True
Is signature of Bob? False
Is signature of Bob? False


## Creating network providers

It's recommended to use the `multiversx_sdk_network_providers` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [95]:
from multiversx_sdk.network_providers import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.multiversx.com")

Creating a Proxy provider:

In [96]:
from multiversx_sdk.network_providers import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.multiversx.com")

## Fetching network parameters

In order to fetch network parameters, do as follows:

In [97]:
config = provider.get_network_config()

print("Chain ID:", config.chain_id)
print("Min gas price:", config.min_gas_price)

Chain ID: D
Min gas price: 1000000000


## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [98]:
account_on_network = provider.get_account(alice)

print("Nonce:", account_on_network.nonce)
print("Balance:", account_on_network.balance)

Nonce: 848
Balance: 1544458693139999963


When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [99]:
from multiversx_sdk.core import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign & broadcast the transaction(s).

For further reference, please see [nonce management](https://docs.multiversx.com/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction:

In [100]:
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

tx = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx))

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)

Transaction hash: 4fcbadad2ecf8e505cfbf8e7ef4a05bc516c76efe01bdd963cefc052d6d66c90


Broadcast multiple transactions:

In [101]:
tx_1 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice.to_bech32(),
    receiver=alice.to_bech32(),
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_1))
tx_2.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_2))
tx_3.signature = signer.sign(transaction_computer.compute_bytes_for_signing(tx_3))

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)

Transactions hashes: (3, {'0': '4fcbadad2ecf8e505cfbf8e7ef4a05bc516c76efe01bdd963cefc052d6d66c90', '1': '1bbb818d7f6a1b5304af5a78216e8b59331faa84f0bfcc3561ff6e0ff8204625', '2': 'dfc4df48dbffebf8efde7e3b5d5be1d6c5e3914ceb988a5eeb164d676e588087'})


Now let's fetch a previously-broadcasted transaction:

In [102]:
tx_on_network = provider.get_transaction("9270a6879b682a7b310c659f58b641ccdd5f083e5633669817130269e5b0939b", with_process_status=True)
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)

Status: success
Is completed: True
